In [12]:
## data.py
import pandas as pd
import numpy as np
import torch

## Deep Ritz Method for Solving PDEs  
The Deep Ritz Method is a neural network-based approach for solving partial differential equations (PDEs) by reformulating the problem as a variational problem. This method leverages the expressive power of deep neural networks to approximate the solution of PDEs.

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [14]:
def sample_interior(n_samples):
    """Sample points in the interior of the domain [0, 1] x [0, 1]."""
    x = np.random.uniform(0, 1, (n_samples, 1))
    y = np.random.uniform(0, 1, (n_samples, 1))
    return torch.tensor(np.hstack((x, y)), dtype=torch.float32).to(device)

def sample_boundary(n_samples, require_normals=False):
    """
    Sample points and normal vectorson the boundary of the domain [0, 1] x [0, 1].
    Returns:
        points: Tensor of shape (n_samples, 2) containing boundary points.
        normals: Tensor of shape (n_samples, 2) containing normal vectors at the boundary points.
    """
    n_samples_per_side = n_samples // 4
    # Four sides of the square
    left = np.stack((np.zeros((n_samples_per_side, 1)), np.random.uniform(0, 1, (n_samples_per_side, 1))), axis=1)
    right = np.stack((np.ones((n_samples_per_side, 1)), np.random.uniform(0, 1, (n_samples_per_side, 1))), axis=1)
    bottom = np.stack((np.random.uniform(0, 1, (n_samples_per_side, 1)), np.zeros((n_samples_per_side, 1))), axis=1)
    top = np.stack((np.random.uniform(0, 1, (n_samples_per_side, 1)), np.ones((n_samples_per_side, 1))), axis=1)
    
    #stack all boundary points
    points = np.vstack((left, right, bottom, top))
    normals = None
    
    if require_normals:
        #normal vectors 
        normals = np.vstack([
            np.tile([-1, 0], (n_samples_per_side, 1)),  # left
            np.tile([1, 0], (n_samples_per_side, 1)),
            np.tile([0, -1], (n_samples_per_side, 1)),  # bottom
            np.tile([0, 1], (n_samples_per_side, 1))   # top
        ])
    
    return (
        torch.tensor(points, dtype=torch.float32).to(device),
        torch.tensor(normals, dtype=torch.float32).to(device) if require_normals else None
    )
       

def generate_samples(n_interior, n_boundary, require_normals=False):
    """Generate interior and boundary samples."""
    interior_points = sample_interior(n_interior)
    boundary_points, boundary_normals = sample_boundary(n_boundary, require_normals)
    return interior_points, boundary_points, boundary_normals
    
    

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class U_FCN(nn.Module):
    def __init__(self,in__features : int = 2, out_features : int = 1, hidden_dims : list = [8,16,32,32,16,8]):
        super(U_FCN,self).__init__()
        layers = []
        input_dim = in__features
        for h_dim in hidden_dims:
            # layers.append(nn.BatchNorm1d(input_dim))
            layers.append(nn.Linear(input_dim, h_dim))
            layers.append(nn.Tanh())
            # layers.append(nn.Dropout(p=0.2))
            input_dim = h_dim
        layers.append(nn.Linear(input_dim, out_features))
        self.network = nn.Sequential(*layers)
    def forward(self,x):
        return self.network(x)
    
# ---------------biharmonic operator ---------------

def biharmonic_operator(u : torch.Tensor,x : torch.Tensor):
    """
    Compute Δ²u = Laplacian(Laplacian(u)) using autograd.
    x : torch.Tensor (N,2) with required_grad = True
    return : torch.Tensor (N,1)
    """
    if not x.requires_grad:
        x.requires_grad = True
    grad_u = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(u),create_graph=True)[0]
    u_x = grad_u[:,0:1]
    u_y = grad_u[:,1:2]
    u_xx = torch.autograd.grad(u_x,x,grad_outputs=torch.ones_like(u_x),create_graph=True)[0][:,0:1]
    u_yy = torch.autograd.grad(u_y,x,grad_outputs=torch.ones_like(u_y),create_graph=True)[0][:,1:2]
    lap_u = u_xx + u_yy
    
    grad_lap_u = torch.autograd.grad(lap_u,x,grad_outputs=torch.ones_like(lap_u),create_graph=True)[0]
    lap_u_x = grad_lap_u[:,0:1]
    lap_u_y = grad_lap_u[:,1:2]
    lap_u_xx = torch.autograd.grad(lap_u_x,x,grad_outputs=torch.ones_like(lap_u_x),create_graph=True)[0][:,0:1]
    lap_u_yy = torch.autograd.grad(lap_u_y,x,grad_outputs=torch.ones_like(lap_u_y),create_graph=True)[0][:,1:2]
    
    return lap_u_xx + lap_u_yy


### Penalized Energy Functional (P2)

$$
\mathcal{L}_\lambda(v)
= 
\frac{1}{2}\int_{\Omega} \lvert D^{2} v \rvert^{2}\, dx
\;-\;
\int_{\Omega} f\, v\, dx
\;-\;
\int_{\partial\Omega} g_{2}\, \frac{\partial v}{\partial n}\, ds
\;+\;
\frac{\lambda}{2}\int_{\partial\Omega} (v - g_{1})^{2}\, ds.  

$$

**Where:**

- $D^2 v$ is the Hessian matrix of $v$.  
- $|D^2 v|^2 = \sum_{i,j} \left( \frac{\partial^2 v}{\partial x_i \partial x_j} \right)^2$.


In [17]:
def compute_hassian( u : torch.Tensor, x : torch.Tensor):
    """
    Compute the Hessian matrix of u with respect to x.
    u : torch.Tensor (N,1)
    x : torch.Tensor (N,2) with required_grad = True
    return : torch.Tensor (N,2,2)
    """
    if not x.requires_grad:
        x.requires_grad = True
    grad_u = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(u),create_graph=True)[0]
    u_x = grad_u[:,0:1]
    u_y = grad_u[:,1:2]
    
    u_xx = torch.autograd.grad(u_x,x,grad_outputs=torch.ones_like(u_x),create_graph=True)[0][:,0:1]
    u_xy = torch.autograd.grad(u_x,x,grad_outputs=torch.ones_like(u_x),create_graph=True)[0][:,1:2]
    u_yx = torch.autograd.grad(u_y,x,grad_outputs=torch.ones_like(u_y),create_graph=True)[0][:,0:1]
    u_yy = torch.autograd.grad(u_y,x,grad_outputs=torch.ones_like(u_y),create_graph=True)[0][:,1:2]
    
    hessian = torch.stack([
        torch.cat([u_xx, u_xy], dim=1),
        torch.cat([u_yx, u_yy], dim=1)
    ], dim=1)  # Shape (N, 2, 2)
    
    return hessian

def compute_normal_derivative(u: torch.Tensor, x: torch.Tensor, normals: torch.Tensor):
    """
    Compute the normal derivative of u at boundary points.
    u : torch.Tensor (N,1)
    x : torch.Tensor (N,2) with required_grad = True
    normals : torch.Tensor (N,2)
    return : torch.Tensor (N,1)
    """
    if not x.requires_grad:
        x.requires_grad = True
    grad_u = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(u),create_graph=True)[0]
    normal_derivative = torch.sum(grad_u * normals, dim=1, keepdim=True)
    return normal_derivative

def compute_first_normal_derivative(u: torch.Tensor, x: torch.Tensor, normal: torch.Tensor):
    """
    Compute the first normal derivative ∂u/∂n using autograd.
    u : torch.Tensor (N,1) with required_grad = True
    x : torch.Tensor (N,2)
    normal : torch.Tensor (N,2)
    return : torch.Tensor (N,1)
    """
    if not x.requires_grad:
        x.requires_grad = True
    
    grad_u = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(u),create_graph=True)[0]
    du_dn = torch.sum(grad_u * normal, dim=1, keepdim=True) #TODO we could use dot product here
    
    return du_dn


def compute_second_normal_derivative(u: torch.Tensor, x: torch.Tensor, normal: torch.Tensor):
    """
    Compute the second normal derivative ∂²u/∂n² using autograd.
    u : torch.Tensor (N,1) with required_grad = True
    x : torch.Tensor (N,2)
    normal : torch.Tensor (N,2)
    return : torch.Tensor (N,1)
    """
    if not x.requires_grad:
        x.requires_grad = True
    
    grad_u = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(u),create_graph=True)[0]
    du_dn = torch.sum(grad_u * normal, dim=1, keepdim=True) #TODO we could use dot product here
    
    grad_du_dn = torch.autograd.grad(du_dn,x,grad_outputs=torch.ones_like(du_dn),create_graph=True)[0]
    d2u_dn2 = torch.sum(grad_du_dn * normal, dim=1, keepdim=True) #similarily
    
    return d2u_dn2
    



In [18]:
def true_solution(pts):
    """ 
    pts : torch.Tensor with shape (N,2)
    """
    if not isinstance(pts,torch.Tensor):
        pts = torch.Tensor(pts,dtype = torch.float32)
    pts = pts.to(dtype=torch.float32, device=pts.device)
    C = 1.0/(2*torch.pi**2)
    sol = C * torch.sin(torch.pi*pts[:,0]) * torch.sin(torch.pi*pts[:,1])
     
    return sol.unsqueeze(-1)

def func(pts):
    """ 
    pts : torch.Tensor with shape (N,2)
    """
    if not isinstance(pts,torch.Tensor):
        pts = torch.Tensor(pts,dtype = torch.float32)
    pts = pts.to(dtype=torch.float32, device=pts.device)
    f =  2.0*torch.pi**2 * torch.sin(torch.pi* pts[:,0]) * torch.sin(torch.pi * pts[:,1])
    return f.unsqueeze(-1)


def g1(pts):
    """ 
    g1(pts) = u same as the neural apporximator
    """
    return true_solution(pts)

def g2(pts,normals):
    """ 
    g2(pts,normals) = d2u/dn2
    """
    u = true_solution(pts)
    # or else we can call the compute_second_normal_derivative function
    d2u_dn2 = normals.unsqueeze(-1) @ compute_hassian(u, pts) @ normals.unsqueeze(-1)
    return d2u_dn2.squeeze(-1)

    
    



### Monte–Carlo Approximation of the Penalized Energy (P2)

Let  
- $\{x_i\}_{i=1}^N \subset \Omega$ be interior collocation points,  
- $\{y_j\}_{j=1}^M \subset \partial\Omega$ be boundary collocation points,  
- $|\Omega|$ the measure of the domain,  
- $|\partial\Omega|$ the measure of the boundary.

Then the Monte–Carlo approximation of $\mathcal{L}_\lambda(v)$ is

$$
\mathcal{L}_{\lambda}^{\text{MC}}(v)
=
\frac{1}{2}\frac{|\Omega|}{N}
\sum_{i=1}^N \big|D^2 v(x_i)\big|^2
\;-\;
\frac{|\Omega|}{N}
\sum_{i=1}^N f(x_i)\, v(x_i)
\;-\;
\frac{|\partial\Omega|}{M}
\sum_{j=1}^M g_2(y_j)\, 
\frac{\partial v}{\partial n}(y_j)
\;+\;
\frac{\lambda}{2}\frac{|\partial\Omega|}{M}
\sum_{j=1}^M \left(v(y_j) - g_1(y_j)\right)^2 .
$$


In [19]:
def interior_loss(model, pts):
    """
    Compute the interior loss for the biharmonic equation Δ²u = f.
    model : neural network model
    pts : torch.Tensor (N,2)
    return : torch.Tensor (1,)
    """
    if not pts.requires_grad:
        pts.requires_grad = True
        
    u = model(pts)
    hessian = compute_hassian(u, pts)
    frobenius_norm_squared = torch.sum(hessian**2, dim=(1, 2))
    source_term = func(pts)*u #shape (N,1)
    loss_interior = torch.mean(0.5 * frobenius_norm_squared - source_term.squeeze(-1))
    return loss_interior

def boundary_loss(model, pts, normals):
    """
    Compute the boundary loss for the biharmonic equation with boundary conditions.
    model : neural network model
    pts : torch.Tensor (N,2)
    normals : torch.Tensor (N,2)
    return : torch.Tensor (1,)
    """
    if not pts.requires_grad:
        pts.requires_grad = True    
    u = model(pts)
    u_true = g1(pts)
    loss_bc1 = F.mse_loss(u, u_true)
    
    neumann_term = g2(pts, normals)*compute_first_normal_derivative(u, pts, normals)
    loss_bc2 = torch.mean(neumann_term)
    loss_boundary = loss_bc1 - loss_bc2
    return loss_boundary

In [ ]:
def validation(model, n_interior = 1000):
    """
    Validate the model by computing the L2 error against the true solution.
    model : neural network model
    n_interior : int, number of interior points for validation
    n_boundary : int, number of boundary points for validation
    return : float, L2 error
    """
    interior_pts, boundary_pts, _ = generate_samples(n_interior, n_boundary)
    all_pts = torch.cat([interior_pts, boundary_pts], dim=0)
    
    u_pred = model(all_pts).detach()
    u_true = true_solution(all_pts).detach()
    
    l2_error = torch.sqrt(torch.mean((u_pred - u_true) ** 2)).item()
    return l2_error

In [21]:
import torch
from tqdm import tqdm
def train1(
    model,
    optimizer,
    log_var_interior : torch.nn.Parameter,
    log_var_boundary : torch.nn.Parameter,
    epochs : int = 1000,
    n_interior : int = 1000,
    n_boundary : int = 400,
    print_interval : int = 100,
    save_path : str = None
    ):
    """
    Train the model using the Deep Ritz Method for the biharmonic equation.
    model : neural network model
    optimizer : torch.optim.Optimizer
    log_var_interior : torch.nn.Parameter, log variance for interior loss
    log_var_boundary : torch.nn.Parameter, log variance for boundary loss
    epochs : int, number of training epochs
    n_interior : int, number of interior points per epoch
    n_boundary : int, number of boundary points per epoch
    print_interval : int, interval for printing training progress
    save_path : str, path to save the trained model
    """
    lossses = []
    interior_losses = []
    boundary_losses = []
    device = next(model.parameters()).device
    pbar = tqdm(range(1,epochs + 1), desc="Training", unit="epoch")
    for epoch in pbar:
        model.train()
        interior_pts, boundary_pts, boundary_normals = generate_samples(n_interior, n_boundary, require_normals=True)
        
        optimizer.zero_grad()
        interior_pts.requires_grad = True
        boundary_pts.requires_grad = True        
        loss_interior = interior_loss(model, interior_pts)
        loss_boundary = boundary_loss(model, boundary_pts, boundary_normals)
        
        precision_interior = torch.exp(-log_var_interior)
        precision_boundary = torch.exp(-log_var_boundary)
        #! total loss with uncertainty weighting
        total_loss = 0.5*(precision_interior * loss_interior + log_var_interior + precision_boundary * loss_boundary + log_var_boundary)
        
        total_loss.backward()
        optimizer.step()
        
        lossses.append(total_loss.item())
        interior_losses.append(loss_interior.item())
        boundary_losses.append(loss_boundary.item())
        
        if epoch % print_interval == 0 or epoch == 1:
            # val_error = validation(model)
            pbar.set_postfix({
                'Total Loss': total_loss.item(),
                'Interior Loss': loss_interior.item(),
                'Boundary Loss': loss_boundary.item(),
                'precision_interior': precision_interior.item(),
                'precision_boundary': precision_boundary.item()
            })
        if epoch % (epochs // 10) == 0:
            val_error = validation(model)
            print(f"Validation L2 Error at epoch {epoch}: {val_error}")
            
    

In [22]:
import os 
os.makedirs('biharmonic_DRM',exist_ok=True)

In [23]:
model = U_FCN(in__features=2,out_features=1,hidden_dims=[16,32,64,128,128,64,32,16]).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
log_var_interior = torch.nn.Parameter(torch.tensor(0.0,device= device))
log_var_boundary = torch.nn.Parameter(torch.tensor(0.0,device= device))
optimizer.add_param_group({'params': [log_var_interior, log_var_boundary],'lr' : 1e-5})

In [24]:
losses, interior_losses, boundary_losses = train1(
    model,
    optimizer,
    log_var_interior,
    log_var_boundary,
    epochs = 5000,
    n_interior = 2000,
    n_boundary = 800,
    print_interval = 100,
    save_path = 'biharmonic_DRM/biharmonic_drm_model.pth'
    )

Training:   0%|          | 0/5000 [00:00<?, ?epoch/s]c:\Users\kumar\venvs\py312\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Training:   0%|          | 0/5000 [00:00<?, ?epoch/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1600x1 and 2x16)